In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Carrega a base de dados contendo os registros de importação de fertilizantes
i=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/imp1.csv')

# Carrega tabela auxiliar com o nome das URF
urf=pd.read_csv('~/git/BD/Comexstat/URF.csv',encoding='Latin-1',sep=';')

# Identifica as URF que contenham no nome os nomes dos portos desejados, Aratu e Salvador.
urf_salvador_aratu=urf[(urf.NO_URF.str.contains('ARATU'))|(urf.NO_URF.str.contains('SALVADOR'))].CO_URF

# Filtra importações ligadas as URF identificadas e destinadas ao estado da Bahia
i=i[(i.CO_URF.isin(urf_salvador_aratu))&(i.SG_UF_NCM=='BA')]

i=i[i.CO_VIA==1]

i.drop(columns=['CO_URF','SG_UF_NCM','CO_VIA','CO_UNID','QT_ESTAT','VL_FOB','VL_FRETE','VL_SEGURO'],inplace=True)

# Corrigindo valor errado
CO_ANO_erro=i[i.KG_LIQUIDO==i.KG_LIQUIDO.max()].CO_ANO.values[0]
CO_ANO_corr_anterior=i[i.CO_ANO==CO_ANO_erro+1].CO_ANO.values[0]
CO_ANO_corr_posterior=i[i.CO_ANO==CO_ANO_erro-1].CO_ANO.values[0]
mes_erro=i[i.KG_LIQUIDO==i.KG_LIQUIDO.max()].CO_MES.values[0]
val_corr=i[i.CO_ANO.isin([CO_ANO_corr_anterior,CO_ANO_corr_posterior])].KG_LIQUIDO.mean()
i.loc[i.KG_LIQUIDO==i.KG_LIQUIDO.max(),'KG_LIQUIDO']=val_corr

i=i[i.KG_LIQUIDO>10**6.3]

In [2]:
# Cria coluna com a data completa e no formato apropriado
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1')

In [3]:
# Carrega tabela "chuvas" qe já vem no formato correto
c=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
display(c.head(1))
display(c.tail(1))

c1=c.copy()
for ad in range(1,11):

    ct=c[['CO_ANO','CO_MES','mm']].copy()
    
    ct.CO_MES=ct.CO_MES+ad
    
    for adi in range(1,ad+1):
        ct.loc[ct.CO_MES==12+adi,'CO_ANO']=ct[ct.CO_MES==12+adi]['CO_ANO']+1
        ct.loc[ct.CO_MES==12+adi,'CO_MES']=adi
    
    ct=ct.rename(columns={'mm':'mm'+'_ad_'+str(ad)})
    
    c1=c1.merge(ct)

c=c1.sort_values(by=['CO_ANO','CO_MES'])
c

,CO_ANO,CO_MES,mm
0,2001,12,23.2


,CO_ANO,CO_MES,mm
264,2023,12,78.6


,CO_ANO,CO_MES,mm,mm_ad_1,mm_ad_2,mm_ad_3,mm_ad_4,mm_ad_5,mm_ad_6,mm_ad_7,mm_ad_8,mm_ad_9,mm_ad_10
0,2002,10,19.4,7.2,8.4,1.2,2.1,22.3,17.8,28.0,0.0,0.0,23.2
1,2002,11,57.3,19.4,7.2,8.4,1.2,2.1,22.3,17.8,28.0,0.0,0.0
2,2002,12,237.8,57.3,19.4,7.2,8.4,1.2,2.1,22.3,17.8,28.0,0.0
3,2003,1,39.6,237.8,57.3,19.4,7.2,8.4,1.2,2.1,22.3,17.8,28.0
4,2003,2,122.7,39.6,237.8,57.3,19.4,7.2,8.4,1.2,2.1,22.3,17.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2023,8,14.8,0.0,0.0,11.6,50.5,83.2,19.9,202.1,239.3,286.8,22.9
251,2023,9,0.8,14.8,0.0,0.0,11.6,50.5,83.2,19.9,202.1,239.3,286.8
252,2023,10,38.7,0.8,14.8,0.0,0.0,11.6,50.5,83.2,19.9,202.1,239.3
253,2023,11,39.8,38.7,0.8,14.8,0.0,0.0,11.6,50.5,83.2,19.9,202.1


In [323]:
q=pd.DataFrame()

i_t=i.groupby(by=['CO_ANO','CO_MES']).KG_LIQUIDO.sum()\
.reset_index().sort_values(by=['CO_ANO','CO_MES'])

i_t['date']=pd.to_datetime(i_t.CO_ANO.astype(str)+'/'+i_t.CO_MES.astype(str)+'/1')

# Martiz Esparsa
# As datas são coletadas no arquivo original com objetivo de fazer a matriz esparça das importações 
# tão longa quanto possível de froma a não perder dados de chuva no momento da fundição das tabelas (merge).
date_ini=i.date.min()
date_fim=i.date.max()
ano_min =i.CO_ANO.min()
ano_max =i.CO_ANO.max()

df_anos =pd.DataFrame({'CO_ANO' :np.arange(ano_min,ano_max+1)}).assign(key=0)
df_meses=pd.DataFrame({'CO_MES' :np.arange(1,13)             }).assign(key=0)

cal=df_anos.merge(df_meses,how='outer').drop(columns='key')

cal['date']=pd.to_datetime(cal.CO_ANO.astype(str)+'/'+cal.CO_MES.astype(str)+'/1')
cal=cal[(cal.date>=date_ini)&(cal.date<=date_fim)]

i_t=i_t.merge(cal,how='outer')

i_t[['CO_ANO','CO_MES']]=i_t[['CO_ANO','CO_MES']].astype('category')
i_t=i_t.groupby(by=['CO_ANO','CO_MES']).KG_LIQUIDO.sum().reset_index()
i_t['date']=pd.to_datetime(i_t.CO_ANO.astype(str)+'/'+i_t.CO_MES.astype(str)+'/1')
i_t=i_t[(i_t.date>=date_ini)&(i_t.date<=date_fim)]


i_c=i_t.merge(c)


i_c['date']=pd.to_datetime(i_c.CO_ANO.astype(str)+'/'+i_c.CO_MES.astype(str)+'/1')

i_c.date=i_c.date.astype(int)
i_c.CO_MES=i_c.CO_MES.astype('category')
i_c=pd.get_dummies(i_c)
i_c.drop(columns='CO_ANO', inplace=True)

q_mm_test=pd.DataFrame()
q_s_mm_test=pd.DataFrame()
q_mm_train=pd.DataFrame()
q_s_mm_train=pd.DataFrame()


for md in [None,1,3,5,8,10,15]:

    from sklearn.ensemble import RandomForestRegressor as RFR

    RFR=RFR(max_depth=md)
    
    
    n_rep=50
    
    for trial in range(n_rep):
        
        train, test = tts(i_c, test_size=0.33)
        
        X_train_mm  =train.drop(columns='KG_LIQUIDO')
        X_train_s_mm=train.drop(columns=['KG_LIQUIDO',
                                         'mm', 'mm_ad_1', 'mm_ad_2', 'mm_ad_3', 'mm_ad_4','mm_ad_5',
                                         'mm_ad_6', 'mm_ad_7', 'mm_ad_8', 'mm_ad_9', 'mm_ad_10'])
        
        y_train=train.KG_LIQUIDO
                
        X_test_mm  =test.drop(columns='KG_LIQUIDO')
        X_test_s_mm=test.drop(columns=['KG_LIQUIDO',
                                       'mm', 'mm_ad_1', 'mm_ad_2', 'mm_ad_3', 'mm_ad_4','mm_ad_5',
                                       'mm_ad_6', 'mm_ad_7', 'mm_ad_8', 'mm_ad_9', 'mm_ad_10'])
        
        y_test=test.KG_LIQUIDO
                
        RFR.fit( X_train_mm, y_train)
                
        # predict sobre test
        y_mm_test=pd.DataFrame(y_test).assign(y_pred=RFR.predict(X_test_mm)).rename(columns={'KG_LIQUIDO':'y_test'})
        r_mm_test=pd.DataFrame({'trial':[trial],'r2_score':[metrics.r2_score(y_mm_test.y_test,y_mm_test.y_pred)]})
    
        # predict sobre train
        y_mm_train=pd.DataFrame(y_train).assign(y_pred=RFR.predict(X_train_mm)).rename(columns={'KG_LIQUIDO':'y_train'})
        r_mm_train=pd.DataFrame({'trial':[trial],'r2_score':[metrics.r2_score(y_mm_train.y_train,y_mm_train.y_pred)]})
    
        
        RFR.fit(X_train_s_mm,y_train)
        # predict sobre test
        y_s_mm_test=pd.DataFrame(y_test).assign(y_pred=RFR.predict(X_test_s_mm)).rename(columns={'KG_LIQUIDO':'y_test'})
        r_s_mm_test=pd.DataFrame({'trial':[trial],'r2_score':[metrics.r2_score(y_s_mm_test.y_test,y_s_mm_test.y_pred)]})
    
    
        # predict sobre train
        y_s_mm_train=pd.DataFrame(y_train).assign(y_pred=RFR.predict(X_train_s_mm)).rename(columns={'KG_LIQUIDO':'y_train'})
        r_s_mm_train=pd.DataFrame({'trial':[trial],'r2_score':[metrics.r2_score(y_s_mm_train.y_train,y_s_mm_train.y_pred)]})
    
    
        #q_mm  =pd.concat([q_mm  ,r_mm])
    #               q_s_mm=pd.concat([q_s_mm,r_s_mm])
    
        q_mm_test=pd.concat([q_mm_test  ,r_mm_test])
        q_s_mm_test=pd.concat([q_s_mm_test,r_s_mm_test])
        q_mm_train=pd.concat([q_mm_train  ,r_mm_train])
        q_s_mm_train=pd.concat([q_s_mm_train,r_s_mm_train])
    
        
    q_mm_mean_test  =q_mm_test  .r2_score.mean()
    q_s_mm_mean_test=q_s_mm_test.r2_score.mean()
    
    q_mm_mean_train  =q_mm_train  .r2_score.mean()
    q_s_mm_mean_train=q_s_mm_train.r2_score.mean()
    
    q_mm_std_test  =q_mm_test  .r2_score.std()
    q_s_mm_std_test=q_s_mm_test.r2_score.std()
    
    q_mm_std_train  =q_mm_train  .r2_score.std()
    q_s_mm_std_train=q_s_mm_train.r2_score.std()
    
    from scipy.stats import norm
    z_score_test=(q_mm_mean_test-q_s_mm_mean_test)/q_mm_std_test
    p_value_test=round(norm.cdf(z_score_test),2)
    
    z_score_train=(q_mm_mean_train-q_s_mm_mean_train)/q_mm_std_train
    p_value_train=round(norm.cdf(z_score_train),2)
    
    p_value_dif=p_value_test-p_value_train
    
    r=pd.DataFrame({
        'md':[md],
        '#repetições':[n_rep],
        'p-value_test':[p_value_test],
        'p-value_train':[p_value_train],
        'p-value_dif':[p_value_dif]
    })
    
    q=pd.concat([q,r])

q.sort_values('p-value_dif', ascending=False)

,md,#repetições,p-value_test,p-value_train,p-value_dif
0,None,50,0.56,0.53,0.03
0,1,50,0.53,0.50,0.03
0,3,50,0.57,0.56,0.01
0,5,50,0.59,0.60,-0.01
0,15,50,0.57,0.59,-0.02
0,8,50,0.58,0.61,-0.03
0,10,50,0.57,0.60,-0.03


In [324]:
q_c=q.copy()

In [325]:
q_c.md=q_c.md.astype('str')

In [332]:
q_c.sort_values('p-value_test')

,md,#repetições,p-value_test,p-value_train,p-value_dif
0,1,50,0.53,0.50,0.03
0,None,50,0.56,0.53,0.03
0,3,50,0.57,0.56,0.01
0,10,50,0.57,0.60,-0.03
0,15,50,0.57,0.59,-0.02
0,8,50,0.58,0.61,-0.03
0,5,50,0.59,0.60,-0.01
